<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#동적-크롤링을-통한-로그인" data-toc-modified-id="동적-크롤링을-통한-로그인-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>동적 크롤링을 통한 로그인</a></span></li><li><span><a href="#크롤링" data-toc-modified-id="크롤링-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>크롤링</a></span><ul class="toc-item"><li><span><a href="#id,-제목,-추천수,-날짜,-조회수" data-toc-modified-id="id,-제목,-추천수,-날짜,-조회수-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>id, 제목, 추천수, 날짜, 조회수</a></span></li><li><span><a href="#META-DATA-TEST" data-toc-modified-id="META-DATA-TEST-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>META DATA TEST</a></span></li><li><span><a href="#게시물-+-댓글" data-toc-modified-id="게시물-+-댓글-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>게시물 + 댓글</a></span><ul class="toc-item"><li><span><a href="#CONTENTS-COMMENTS-TEST" data-toc-modified-id="CONTENTS-COMMENTS-TEST-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>CONTENTS COMMENTS TEST</a></span></li></ul></li></ul></li></ul></div>

## 동적 크롤링을 통한 로그인

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime

In [3]:
def login():
    # Chrome WebDriver 초기화
    try:
        driver = webdriver.Chrome()

        # 웹페이지로 이동
        driver.get('http://www.ssodam.com')

            # '닫기' 버튼을 찾기 위해 data-dismiss 속성을 이용
        close_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-dismiss="modal"]'))
        )
        close_button.click()

        # "로그인" 링크를 찾아 클릭
        login_link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, '로그인'))
        )
        login_link.click()

        # 아이디 입력
        id_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.NAME, 'id'))
        )
        id_input.send_keys('kun0107')  # 본인 아이디 입력
        id_input.click()


        pass_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.NAME, 'pass'))
        )
        pass_input.click()
        pass_input.send_keys('Dlrjsghk12@@')

        # 로그인 버튼 클릭
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//input[@type="submit" and @value="로그인"]'))
        )
        login_button.click()

    except Exception as e:
        print("로그인 실패")
        print(e)
        return NULL
    return driver

## 크롤링

### meta data

In [4]:
def get_ids(soup):
    ids = []
    td_tags = soup.find_all('td', class_='mobile-hide')
    for i in range(min(20, len(td_tags))):
        td_tag = td_tags[i]
        id = int(td_tag.find('center').text)
        ids.append(id)

    return ids

In [5]:
def get_titles(soup):
    titles = []
    title_tags = soup.find_all('span', class_='content-title')
    for i in range(min(20, len(title_tags))):
        title = title_tags[i].text.strip()
        titles.append(title)
    return titles

In [6]:
def get_etc(soup):
    values = soup.find_all('td', style='text-align:center;')
    likes = []
    dates = []
    views = []
    for i in range(4, min(84, len(values)), 4):
        like = int(values[i + 1].text.strip())
        date = datetime.strptime(values[i + 2].text.strip(), '%m/%d')
        view = int(values[i + 3].text.strip())
        likes.append(like)
        dates.append(date)
        views.append(view)
    return likes, dates, views

#### handler

In [31]:
def metadata_handler(driver, big_kw, max_page):
    meta_data = {
        "id": [],
        "title": [],
        "likes": [],
        "dates": [],
        "views": [],
    }
    for i in range(1, max_page + 1):
        url = 'https://www.ssodam.com/all/' + str(i) + '?field=content&q=' + big_kw
        driver.get(url)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        ids = get_ids(soup)
        titles = get_titles(soup)
        likes, dates, views = get_etc(soup)
        meta_data["id"].extend(ids)
        meta_data["title"].extend(titles)
        meta_data["likes"].extend(likes)
        meta_data["dates"].extend(dates)
        meta_data["views"].extend(views)
    meta_df = pd.DataFrame(meta_data)
    return meta_df
        

##### meta data test for handler

In [ ]:
driver = login()
url = 'https://www.ssodam.com/all/1?field=content&q=남사친'
time.sleep(1)
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
meta_data = get_posts_metadata(soup)
meta_df = pd.DataFrame(meta_data)
meta_df

,id,title,likes,dates,views
0,911221,남사친 많은 여자만 만나는 병 있음,4,1900-01-22,809
1,1747515,소개팅앱 개고인물의 주저리,8,1900-01-15,368
2,1744610,극보수 유교남들 들어와줘,2,1900-01-11,848
3,1743307,"축구, 농구, 스키, 수영, 배드민턴, 탁구, 볼링 등 운동 잘하는데",1,1900-01-09,199
4,1742695,여사친이랑 신년운세 보러가기,3,1900-01-04,814
5,1740657,남친이 성향자인 걸까요??,6,1900-12-31,836
6,1739237,잘난 사람 가까이 두면 본인한테 이득 아님?,8,1900-12-31,427
7,1739035,여자 모솔 22살이면 답 없음?,-4,1900-12-28,509
8,1737530,이사람 나한테 관심 없는거임??,7,1900-12-27,383
9,1737447,첫 여자친구랑 크리스마스 보내면서 상처만 남음,106,1900-12-27,5831


### contents comments

In [35]:
def get_contents_comments(soup):
    
    # 게시물 내용
    contents = soup.select('.board-content.word-break')[0].text.strip()
    comments_list = []

    # 게시물 댓글
    comments = soup.select('.word-break.comment-content')
    for comment in comments :
        comment_text = comment.text.strip()
        comments_list.append(comment_text)
    comments_num_str = soup.select('.comment-number')[0].text
    comments_num = int(''.join(filter(str.isdigit, comments_num_str)))
    return contents, comments_list, comments_num

#### handler

In [41]:
def contents_handler(driver, ids):
    contents_data = {
        "id" : [],
        "contents": [],
        "comments" : [],
        "comments_num" : []
    }    
    for ID in ids:
        url = 'https://www.ssodam.com/content/' + str(ID)
        driver.get(url)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        contents, comments_list, comments_num = get_contents_comments(soup)
        contents_data["id"].append(ID)
        contents_data["contents"].append(contents)
        contents_data["comments_num"].append(comments_num)
        contents_data["comments"].append(comments_list)
    contents_df = pd.DataFrame(contents_data)     
    return contents_df

        

##### test for comments contents handler

In [18]:
driver = login()
url = 'https://www.ssodam.com/content/1645555'
time.sleep(1)
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
contents_data = {
    "id": [],
    "contents": [],
    "comments_num": []
}
comments_data = {
    "id": [],
    "comments": [],
}
contents, comments_list, comments_num = get_contents_comments(soup)

contents, comments_list, comments_num
contents_data["id"].append(1645555)
contents_data["contents"].append(contents)
contents_data["comments_num"].append(comments_num)
contents_df = pd.DataFrame(contents_data)     
comments_data["id"].append(1645555)
comments_data["comments"].append(comments_list)
comments_df = pd.DataFrame(comments_data)     

In [122]:
driver = login()
url = 'https://www.ssodam.com/content/1645555?prev=1&prev_content=/board/6'
time.sleep(0.2)
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
contents, comments_list, comments_num = get_contents_comments(soup)
contents_data = {
}
contents_data["id"].append(1645555)
contents_data["contents"].append(contents)
contents_data["comments_num"].append(comments_num)
contents_data["comments"].append(comments_list)
contents_df = pd.DataFrame(contents_data)     


## Main

### login

In [ ]:
driver = login()

### meta data

In [36]:
meta_df = metadata_handler(driver, '남사친', 3)
meta_df

,id,title,likes,dates,views
0,911221,남사친 많은 여자만 만나는 병 있음,4,1900-01-22,809
1,1747515,소개팅앱 개고인물의 주저리,8,1900-01-15,368
2,1744610,극보수 유교남들 들어와줘,2,1900-01-11,848
3,1743307,"축구, 농구, 스키, 수영, 배드민턴, 탁구, 볼링 등 운동 잘하는데",1,1900-01-09,199
4,1742695,여사친이랑 신년운세 보러가기,3,1900-01-04,814
5,1740657,남친이 성향자인 걸까요??,6,1900-12-31,836
6,1739237,잘난 사람 가까이 두면 본인한테 이득 아님?,8,1900-12-31,427
7,1739035,여자 모솔 22살이면 답 없음?,-4,1900-12-28,509
8,1737530,이사람 나한테 관심 없는거임??,7,1900-12-27,383
9,1737447,첫 여자친구랑 크리스마스 보내면서 상처만 남음,106,1900-12-27,5831


### Extract preferred ids

In [37]:
ids = meta_df[meta_df['views'] > 1000]['id']
ids

9     1737447
13    1732856
15    1732327
21    1724173
23    1722225
24    1720450
26    1717002
27    1712232
28    1711794
29    1706059
32    1701604
34    1698295
35    1696594
36    1691111
37    1690602
39    1690380
43    1677337
51    1666729
53    1665199
54    1663740
Name: id, dtype: int64

### contents

In [42]:
contents_df= contents_handler(driver, ids)
contents_df

,id,contents,comments,comments_num
0,1737447,그냥 친한 남사친이고 얼마전에 단 둘이 만남. 밥 먹고 나왔는데 뭔가 둘이 할 게 ...,"[넌 관심있는 거 아냐?, 롤하고 싶었을텐데, 아 ㅋㅋㅋㅋ, 찐사랑, 관심없으면 피...",7
1,1732856,"내 친구 얘기인데내 남사친은 A를 좋아함. A는 B(여자, A가 동성애자임)를 좋아...","[A쓰레기네, 내가 계속 A 좀 별로다. 나였으면 거리 뒀을 거다. 하는데 남사친이...",2
2,1732327,여사친이 이번주 금욜부터 다음주 월요일까지 자기 친구 셋이랑 오사카 여행이거든. 그...,"[니 감정은?, 나도 얘 이성적으로 좋아, ㄱㄱ, ㄱㄱ, 연게에 올려라 새키야., ...",11
3,1724173,남사친도 주위에 꽤 잇는것 같고...나도 그냥 친구중하나일까,"[쎄하면 맞다, 오빠야? 아닌데, 4살 오빠야? 그럼 아니야 이성적 호감이야, 경영...",6
4,1722225,대쉬 받기만 하고 직접 대쉬하는 이런 적이 처음이기도 해서.. 어떻게 남자분이 받아...,"[어디서 만났는지에 따라 방법이 다른데, 진짜 여자 대쉬 남자 입장 부담은 어디 학...",5
5,1720450,내가 일홉을 안가봐서 잘 모르는데 다들 그냥 보내주나?난 그냥 대학헌포라고 생각했어...,"[난 우리과 일홉때 스태프들이 친구들이어서, 동성친구랑 합석 없이 술만 먹긴 함.....",15
6,1717002,내가 뒤틀린 애정관을 가진 사람이라나도 집착 받는게 좋고 상대방에게도 집착을 하게 ...,[불안할 수 있겠다~ 정신없이 바빠지면 확실히 없어지긴 하더라 ...ㅎㅎㅎ 그리공 ...,26
7,1712232,내 취준 도와준다면서 여친있는 남사친이 매일 연락옴...카톡은 느린데 매일 와 답 ...,[움 그냥 진짜 친절인 사람이 있긴하더라고 나도 비슷하게 아빠를 자처하면서 도와주는...,12
8,1711794,남사친한테도 몸매 좋다는 말 들어보고 필테 쌤 한테도 듣고 비율 좋다는 말 엄청 듣...,"[몸매 어필 오지는데키크고 마른데 나올데나오면 환장함, 엄청 어필되는데못생기지만 않...",16
9,1706059,"난 여자고 친한 남사친 두명이 있음(20대 후반임)셋이서 자주 노는데 남자A, B라...",[ㅋㅋㅋㅋㅋㅋ A는 너 좋아하는 거 맞고 계속 간 보는 중인 거... 니가 미끼를 ...,11
